In [1]:
# -*- coding: utf-8 -*-

## standard packages
import numpy as np
import os
import datetime

## Config Parster for initiation
import configparser

## HTTP scraping packages
from lxml import html
import requests

#import httplib2
import urllib

from bs4 import BeautifulSoup, SoupStrainer
import json

## Database
import sqlite3

import src.utils as utils

In [2]:
pagesConfig = configparser.ConfigParser()
pagesConfig.read("config/siteSpecifics.ini")

for section in pagesConfig.sections():
    print(section)
    #for option in pagesConfig.options(section):
        #print(option, pagesConfig.get(section, option))
    #    print (section, option)

WebPages
additionalAdInfo
dbInit


In [3]:
searchPage = pagesConfig.get('WebPages', 'mobile.de')
print(searchPage)

http://suchen.mobile.de/fahrzeuge/search.html?scopeId=C&damageUnrepaired=NO_DAMAGE_UNREPAIRED&scopeId=C&isSearchRequest=true&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING&pageNumber=


In [ ]:
http://suchen.mobile.de/fahrzeuge/search.html?damageUnrepaired=NO_DAMAGE_UNREPAIRED&isSearchRequest=true&scopeId=C&scopeId=C&
    sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING

In [62]:
searchPage = "http://suchen.mobile.de/fahrzeuge/search.html?scopeId=C&damageUnrepaired=NO_DAMAGE_UNREPAIRED&scopeId=C&isSearchRequest=true&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING"
#payload = {'scopeId': ['C', 'C'],
#           'damageUnrepaired': 'NO_DAMAGE_UNREPAIRED',
#           'isSearchRequest': 'true',
#           'sortOption.sortBy': 'creationTime',
#           'sortOption.sortOrder': 'DESCENDING'}
payload=dict()
dataIDs = list()

for factor in range(100):
    maxPrice = factor*1000+1000
    minPrice = factor*1000
    
    payload['maxPrice'] = maxPrice
    payload['minPrice'] = minPrice
    for i in range(50):
        #print(i)
        payload['pageNumber'] = i+1

        req = requests.get(searchPage, params=payload)
        #print(req.url)
        soup = BeautifulSoup(req.text, "lxml")
        for link in soup.find_all('a'):
            if link.has_attr('data-ad-id'):
                #print(link['data-ad-id'])
                if link['data-ad-id'] in dataIDs:
                    #print(link['data-ad-id'], "already existing")
                    pass
                else:
                    dataIDs.append(link['data-ad-id'])

    print(factor, minPrice, maxPrice, len(dataIDs))

0 0 1000 997
1 1000 2000 1950
2 2000 3000 2896
3 3000 4000 3863
4 4000 5000 4827
5 5000 6000 5796
6 6000 7000 6752
7 7000 8000 7718
8 8000 9000 8690
9 9000 10000 9673


KeyboardInterrupt: 

In [68]:
len(dataIDs)

9886

In [57]:
#http = urllib
#response = http.request.urlopen(searchPage)

dataIDs = list()
        
soup = BeautifulSoup(req.content, "lxml") # put search page in beautiful soup strainer,
dataIDs = list()
for link in soup.find_all('a'): # save all links in the search page that have an ad-id (i.e. they are not an ad)\n",
    if link.has_attr('data-ad-id'):
        print(link['data-ad-id'])
        print(link['href'])
        print(link.get_text('href')[:150] + " ...")
        
        onPos = link.get_text('href').find('online seit ')
        onlineSince = link.get_text('href')[onPos+12 : onPos+22]
        
        print(onlineSince)
        print()
        dataIDs.append([link['data-ad-id'], link['href'], link.get_text('href'), onlineSince])


247933152
https://suchen.mobile.de/fahrzeuge/details.html?id=247933152&damageUnrepaired=NO_DAMAGE_UNREPAIRED&isSearchRequest=true&pageNumber=50&scopeId=C&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING&fnai=prev
NeuhrefVolkswagen Touran 2.0 TDI BlueMotion TechhrefInserat online seit 25.07.2017 16:26href9.885 €hrefEZ 07/2011, 168.010 km, 103 kW (140 PS)hrefVan  ...
25.07.2017

247933151
https://suchen.mobile.de/fahrzeuge/details.html?id=247933151&damageUnrepaired=NO_DAMAGE_UNREPAIRED&isSearchRequest=true&pageNumber=50&scopeId=C&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING
NeuhrefFord Kuga 2.0 TDCi 4x4 TitaniumhrefInserat online seit 25.07.2017 16:26href17.885 €hrefEZ 10/2014, 88.280 km, 103 kW (140 PS)hrefGeländewagen / ...
25.07.2017

247933150
https://suchen.mobile.de/fahrzeuge/details.html?id=247933150&damageUnrepaired=NO_DAMAGE_UNREPAIRED&isSearchRequest=true&pageNumber=50&scopeId=C&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING
 href

In [43]:
dataIDs[0][3]

IndexError: list index out of range

In [8]:
utils.getInfoFromPage(dataIDs[0][1])

{'ad': {'condition': 'USED',
  'currency': 'EUR',
  'price': '29998',
  'specifics': {'make': 'Audi', 'model': 'Q7'}},
 'adCondition': 'USED',
 'adCurrency': 'EUR',
 'adFirstRegMonth': '3',
 'adFirstRegYear': '2013',
 'adId': 247061537,
 'adImage': 'http://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/El8AAOSwyltZXkpt/$_23.JPG',
 'adImageBase64': 'aHR0cDovL2kuZWJheWltZy5jb20vMDAvcy9NVEl3TUZneE5qQXcvei9FbDhBQU9Td3lsdFpYa3B0LyRfMjMuSlBH',
 'adPrice': '29998',
 'adSegment': 'Car',
 'adSpecificsFuel': 'DIESEL',
 'adSpecificsHsn': '588',
 'adSpecificsMake': 'Audi',
 'adSpecificsMakeId': 1900,
 'adSpecificsMakeModel': 'Audi Q7',
 'adSpecificsMakeModelBase64': 'QXVkaSBRNw==',
 'adSpecificsModel': 'Q7',
 'adSpecificsModelDescription': 'Q7 3.0 TDI DPF quatt. tiptr. 7-Sitzer/Scheckheft',
 'adSpecificsModelId': 15,
 'adSpecificsTsn': 'ARQ',
 'airbag': 'Front-, Seiten- und weitere Airbags',
 'category': 'Limousine',
 'climatisation': 'Klimaautomatik',
 'countryVersion': 'No Information',
 'cubicCapacity': '2.

In [159]:
dataIDs[1]

['241090027',
 'http://suchen.mobile.de/fahrzeuge/details.html?id=241090027&damageUnrepaired=NO_DAMAGE_UNREPAIRED&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING&scopeId=C&isSearchRequest=true&pageNumber=1',
 ' href15hrefNeuhrefAudi A5 Sportback 3.0 TDI quattro S.line Kame Xe LedehrefInserat online seit 12.04.2017 15:11href20.990 €href19,00% MwSt.hrefEZ 11/2013, 151.963\xa0km, 180\xa0kW\xa0(245\xa0PS)hrefSportwagen / Coupé, Diesel, Automatik, 4/5 Türenhref≈\u20095,8\u2009l/100km (komb.), ≈\u2009154\u2009g CO₂/km (komb.)hrefCar Center Ganderkesee href(href18href)hrefDE-27777 Ganderkesee, HändlerhrefFinanzierunghref, hrefVersicherunghrefGeparkthrefParkenhrefKontakt',
 '12.04.2017']

In [162]:
now = datetime.datetime.now()
str(now.date())

'2017-04-12'

In [154]:
allInfos['description'] = div.get_text(separator='\n')
allInfos['firstSeen'] = dataIDs[0][3]
allInfos['lastSeen'] = str(datetime.datetime.now().date)
allInfos

{'ad': {'condition': 'USED',
  'currency': 'EUR',
  'price': '12999',
  'specifics': {'make': 'Toyota', 'model': 'RAV 4'}},
 'adCondition': 'USED',
 'adCurrency': 'EUR',
 'adFirstRegMonth': '11',
 'adFirstRegYear': '2011',
 'adId': 238600200,
 'adImage': 'http://i.ebayimg.com/00/s/OTAwWDE2MDA=/z/SskAAOSwol5YypUC/$_23.JPG',
 'adImageBase64': 'aHR0cDovL2kuZWJheWltZy5jb20vMDAvcy9PVEF3V0RFMk1EQT0vei9Tc2tBQU9Td29sNVl5cFVDLyRfMjMuSlBH',
 'adPrice': '12999',
 'adSegment': 'Car',
 'adSpecificsFuel': 'DIESEL',
 'adSpecificsMake': 'Toyota',
 'adSpecificsMakeId': 24100,
 'adSpecificsMakeModel': 'Toyota RAV 4',
 'adSpecificsMakeModelBase64': 'VG95b3RhIFJBViA0',
 'adSpecificsModel': 'RAV 4',
 'adSpecificsModelDescription': 'RAV 4 Executive Automatik 4x4',
 'adSpecificsModelId': 28,
 'adSpecificsVin': 'JTMBB31V90D088148',
 'airbag': 'Front-, Seiten- und weitere Airbags',
 'category': 'Geländewagen / Pickup',
 'climatisation': 'Klimaautomatik',
 'countryVersion': 'No Information',
 'cubicCapacity': '

In [157]:
for key, value in allInfos.items():
    print( key, value, type(value) )

ad {'price': '12999', 'currency': 'EUR', 'condition': 'USED', 'specifics': {'make': 'Toyota', 'model': 'RAV 4'}} <class 'dict'>
isDealer True <class 'bool'>
dealerCountry DE <class 'str'>
dealerZip 12351 <class 'str'>
dealerCity Berlin <class 'str'>
adId 238600200 <class 'int'>
adSegment Car <class 'str'>
adCondition USED <class 'str'>
adFirstRegYear 2011 <class 'str'>
adFirstRegMonth 11 <class 'str'>
adPrice 12999 <class 'str'>
adCurrency EUR <class 'str'>
adImage http://i.ebayimg.com/00/s/OTAwWDE2MDA=/z/SskAAOSwol5YypUC/$_23.JPG <class 'str'>
adImageBase64 aHR0cDovL2kuZWJheWltZy5jb20vMDAvcy9PVEF3V0RFMk1EQT0vei9Tc2tBQU9Td29sNVl5cFVDLyRfMjMuSlBH <class 'str'>
adSpecificsMake Toyota <class 'str'>
adSpecificsMakeId 24100 <class 'int'>
adSpecificsModel RAV 4 <class 'str'>
adSpecificsModelId 28 <class 'int'>
adSpecificsMakeModel Toyota RAV 4 <class 'str'>
adSpecificsMakeModelBase64 VG95b3RhIFJBViA0 <class 'str'>
adSpecificsModelDescription RAV 4 Executive Automatik 4x4 <class 'str'>
adSpec

## initiate Database

In [69]:
%pwd

'H:\\GitHub_Projects\\UsedCarStats'

In [70]:
attributes = dict()
for option in pagesConfig.options('dbInit'):
    attributes[option] = pagesConfig.get('dbInit', option)
    

In [152]:
s = ''
for key, val in attributes.items():
    s += key + ' ' + val + ', '
sql_create_command = '''CREATE TABLE car ( %s )''' %s[:-2]

print(sql_create_command)

CREATE TABLE car ( adid INTEGER PRIMARY KEY, adcondition VARCHAR, adcurrency VARCHAR, adfirstregmonth INT, adfirstregyear INT, adprice FLOAT, adsegment VARCHAR, adspecificsfuel VARCHAR, adspecificshsn INT, adspecificsmake VARCHAR, adspecificsmakeid VARCHAR, adspecificsmakemodel VARCHAR, adspecificsmodel VARCHAR, adspecificsmodeldescription TEXT, adspecificsmodelid VARCHAR, dealercity VARCHAR, dealercountry VARCHAR, dealerzip INT, featurevariants TEXT, isdealer BOOLEAN, sellertype VARCHAR, usersipaddress VARCHAR, category VARCHAR, countryversion VARCHAR, mileage VARCHAR, cubiccapacity VARCHAR, power VARCHAR, fuel VARCHAR, fuelconsumption VARCHAR, emission VARCHAR, numseats INT, doorcount VARCHAR, transmission VARCHAR, emissionssticker VARCHAR, firstregistration VARCHAR, numberofpreviousowners INT, hu VARCHAR, climatisation VARCHAR, parkassist VARCHAR, airbag VARCHAR, manufacturercolorname VARCHAR, color VARCHAR, interior VARCHAR, features TEXT, description TEXT, firstseen DATE, lastseen

In [ ]:
connection = sqlite3.connect('src/dataBase/carStats.db')
cursor = connection.cursor()

# delete 
#cursor.execute("""DROP TABLE car;""")

#cursor.execute(sql_create_command)

# never forget this, if you want the changes to be saved:
#connection.commit()

#connection.close()

In [236]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
result = cursor.fetchall()
result

[('car',)]

In [238]:
cursor.close

'car'

In [232]:
#cursor.execute(sql_create_command)
cursor.execute('''SELECT * FROM car''')
cursor.fetchall()

[(140655129,
  'USED',
  'EUR',
  6,
  1999,
  3940.0,
  'Car',
  'PETROL',
  None,
  'Smart',
  '23000',
  'Smart ForTwo',
  'ForTwo',
  'smart BRABUS LM-FELGEN / LEDER / GLASDACH',
  '4',
  'Köln - Poll',
  'DE',
  51105,
  None,
  'True',
  'DEALER',
  '84.153.140.0',
  'Kleinwagen',
  'No Information',
  '72.050\xa0km',
  '599\xa0cm³',
  '40\xa0kW\xa0(54\xa0PS)',
  'Benzin',
  None,
  None,
  2,
  '2/3',
  'Automatik',
  '4 (Grün)',
  '06/1999',
  'No Information',
  '09/2013',
  'Klimaanlage',
  'No Information',
  'Front-Airbags',
  'No Information',
  None,
  'Vollleder, Schwarz',
  'Ausstattung, ABS, CD-Spieler, Elektr. Fensterheber, Elektr. Wegfahrsperre, Leichtmetallfelgen, Servolenkung, Tuner/Radio, Zentralverriegelung',
  'BRABUS 15 Zoll Leichtmetallfelgen\nneue Ganzjahresreifen 185/55R15 montiert\nCarbonfolie beklebt\nSportendschalldämpfer\nEdelstahl Heckbügelrohr\nPanoramaglasdach\nSchaltwippen am Lenkrad\nKlimaanlage\nHalbautomatikgetriebe\nZV mit Fernbedienung\nFront-Ai

In [208]:
pageInfo = utils.getInfoFromPage('https://suchen.mobile.de/fahrzeuge/details.html?id=' + dataIDs[0])


In [209]:
def insertIntoDB(dictionary, superKey=''):
    columns = ''
    values = ''
    for key, val in dictionary.items():
        if type(val) == dict:
            c, v = insertIntoDB(val, superKey=superKey + key)
            columns += c
            values += v
        elif key.lower() in attributes.keys():
            columns += superKey + key + ', '
            values += '"' + str(val) + '"' + ', '


    
    return columns, values

columns, values = insertIntoDB(pageInfo)
    
sql_insert_command = '''INSERT INTO car ( %s ) VALUES ( %s );''' %(columns[:-2], values[:-2])

print(sql_insert_command)

INSERT INTO car ( isDealer, dealerCountry, dealerZip, dealerCity, adId, adSegment, adCondition, adFirstRegYear, adFirstRegMonth, adPrice, adCurrency, adSpecificsMake, adSpecificsMakeId, adSpecificsModel, adSpecificsModelId, adSpecificsMakeModel, adSpecificsModelDescription, adSpecificsFuel, usersIpAddress, sellerType, category, countryVersion, mileage, cubicCapacity, power, fuel, numSeats, doorCount, transmission, emissionsSticker, firstRegistration, numberOfPreviousOwners, hu, climatisation, parkAssist, airbag, manufacturerColorName, interior, features, description ) VALUES ( "True", "DE", "51105", "Köln - Poll", "140655129", "Car", "USED", "1999", "6", "3940", "EUR", "Smart", "23000", "ForTwo", "4", "Smart ForTwo", "smart BRABUS LM-FELGEN / LEDER / GLASDACH", "PETROL", "84.153.140.0", "DEALER", "Kleinwagen", "No Information", "72.050 km", "599 cm³", "40 kW (54 PS)", "Benzin", "2", "2/3", "Automatik", "4 (Grün)", "06/1999", "No Information", "09/2013", "Klimaanlage", "No Information",

In [223]:
cursor.execute(sql_insert_command)
cursor.fetchall()

[]

In [224]:
#cursor.execute("DELETE FROM car;")
cursor.execute("SELECT * FROM car;") 
#cursor.execute("""DROP TABLE car;""")

result = cursor.fetchall()
print(result)

[(140655129, 'USED', 'EUR', 6, 1999, 3940.0, 'Car', 'PETROL', None, 'Smart', '23000', 'Smart ForTwo', 'ForTwo', 'smart BRABUS LM-FELGEN / LEDER / GLASDACH', '4', 'Köln - Poll', 'DE', 51105, None, 'True', 'DEALER', '84.153.140.0', 'Kleinwagen', 'No Information', '72.050\xa0km', '599\xa0cm³', '40\xa0kW\xa0(54\xa0PS)', 'Benzin', None, None, 2, '2/3', 'Automatik', '4 (Grün)', '06/1999', 'No Information', '09/2013', 'Klimaanlage', 'No Information', 'Front-Airbags', 'No Information', None, 'Vollleder, Schwarz', 'Ausstattung, ABS, CD-Spieler, Elektr. Fensterheber, Elektr. Wegfahrsperre, Leichtmetallfelgen, Servolenkung, Tuner/Radio, Zentralverriegelung', 'BRABUS 15 Zoll Leichtmetallfelgen\nneue Ganzjahresreifen 185/55R15 montiert\nCarbonfolie beklebt\nSportendschalldämpfer\nEdelstahl Heckbügelrohr\nPanoramaglasdach\nSchaltwippen am Lenkrad\nKlimaanlage\nHalbautomatikgetriebe\nZV mit Fernbedienung\nFront-Airbags\nZentralverriegelung inkl. Fernbedienung\nPioneer Radio CD-Spieler mit MP3\nColorv

In [226]:
for i in cursor.description:
    print (i[0])

adid
adcondition
adcurrency
adfirstregmonth
adfirstregyear
adprice
adsegment
adspecificsfuel
adspecificshsn
adspecificsmake
adspecificsmakeid
adspecificsmakemodel
adspecificsmodel
adspecificsmodeldescription
adspecificsmodelid
dealercity
dealercountry
dealerzip
featurevariants
isdealer
sellertype
usersipaddress
category
countryversion
mileage
cubiccapacity
power
fuel
fuelconsumption
emission
numseats
doorcount
transmission
emissionssticker
firstregistration
numberofpreviousowners
hu
climatisation
parkassist
airbag
manufacturercolorname
color
interior
features
description
firstseen
lastseen


In [108]:
connection.close()